In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("HogwartsAnalysis").getOrCreate()


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/11 08:48:20 WARN Utils: Your hostname, KSHITISH, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/08/11 08:48:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/11 08:48:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/08/11 08:48:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/08/11 08:48:23 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [25]:
data = [
    ("Gryffindor", 1, 80, "Harry Potter"),
    ("Slytherin", 1, 60, "Draco Malfoy"),
    ("Ravenclaw", 1, 45, "Luna Lovegood"),
    ("Hufflepuff", 1, 30, "Cedric Diggory"),
    ("Gryffindor", 2, 90, "Hermione Granger"),
    ("Slytherin", 2, 70, "Pansy Parkinson"),
    ("Ravenclaw", 2, 55, "Cho Chang"),
    ("Hufflepuff", 2, 65, "Hannah Abbott"),
    ("Gryffindor", 3, 20, "Ron Weasley"),
    ("Slytherin", 3, 85, "Blaise Zabini")
]

columns = ["house", "year", "points", "student"]
df = spark.createDataFrame(data, columns)
df.show()

+----------+----+------+----------------+
|     house|year|points|         student|
+----------+----+------+----------------+
|Gryffindor|   1|    80|    Harry Potter|
| Slytherin|   1|    60|    Draco Malfoy|
| Ravenclaw|   1|    45|   Luna Lovegood|
|Hufflepuff|   1|    30|  Cedric Diggory|
|Gryffindor|   2|    90|Hermione Granger|
| Slytherin|   2|    70| Pansy Parkinson|
| Ravenclaw|   2|    55|       Cho Chang|
|Hufflepuff|   2|    65|   Hannah Abbott|
|Gryffindor|   3|    20|     Ron Weasley|
| Slytherin|   3|    85|   Blaise Zabini|
+----------+----+------+----------------+



In [26]:
from pyspark.sql.functions import col,sum,avg
df_filter = df.filter(col("points") > 50)
df_filter.show()
# df_filter.printSchema()

+----------+----+------+----------------+
|     house|year|points|         student|
+----------+----+------+----------------+
|Gryffindor|   1|    80|    Harry Potter|
| Slytherin|   1|    60|    Draco Malfoy|
|Gryffindor|   2|    90|Hermione Granger|
| Slytherin|   2|    70| Pansy Parkinson|
| Ravenclaw|   2|    55|       Cho Chang|
|Hufflepuff|   2|    65|   Hannah Abbott|
| Slytherin|   3|    85|   Blaise Zabini|
+----------+----+------+----------------+



In [27]:
result_df = (
    df_filter.groupBy("house","year")
    .agg(sum(col("points")).alias("total_sum"))
    .orderBy(col("year"),col("total_sum").desc()))

result_df.show()

+----------+----+---------+
|     house|year|total_sum|
+----------+----+---------+
|Gryffindor|   1|       80|
| Slytherin|   1|       60|
|Gryffindor|   2|       90|
| Slytherin|   2|       70|
|Hufflepuff|   2|       65|
| Ravenclaw|   2|       55|
| Slytherin|   3|       85|
+----------+----+---------+



In [28]:
data = [
    ("Aragorn", "Human", 10, 2, "Helms Deep"),
    ("Legolas", "Elf", 15, 0, "Helms Deep"),
    ("Gimli", "Dwarf", 8, 3, "Helms Deep"),
    ("Frodo", "Hobbit", 2, 1, "Moria"),
    ("Sam", "Hobbit", 4, 2, "Moria"),
    ("Gandalf", "Wizard", 12, 1, "Moria"),
    ("Boromir", "Human", 7, 4, "Amon Hen"),
    ("Legolas", "Elf", 20, 0, "Amon Hen"),
    ("Aragorn", "Human", 9, 2, "Amon Hen")
]

columns = ["name", "race", "enemies_defeated", "injuries", "battle"]
df = spark.createDataFrame(data, columns)
df.show()

+-------+------+----------------+--------+----------+
|   name|  race|enemies_defeated|injuries|    battle|
+-------+------+----------------+--------+----------+
|Aragorn| Human|              10|       2|Helms Deep|
|Legolas|   Elf|              15|       0|Helms Deep|
|  Gimli| Dwarf|               8|       3|Helms Deep|
|  Frodo|Hobbit|               2|       1|     Moria|
|    Sam|Hobbit|               4|       2|     Moria|
|Gandalf|Wizard|              12|       1|     Moria|
|Boromir| Human|               7|       4|  Amon Hen|
|Legolas|   Elf|              20|       0|  Amon Hen|
|Aragorn| Human|               9|       2|  Amon Hen|
+-------+------+----------------+--------+----------+



In [29]:
df_filter_2 = df.filter(col("enemies_defeated") > 5)
df_filter_2.show()

+-------+------+----------------+--------+----------+
|   name|  race|enemies_defeated|injuries|    battle|
+-------+------+----------------+--------+----------+
|Aragorn| Human|              10|       2|Helms Deep|
|Legolas|   Elf|              15|       0|Helms Deep|
|  Gimli| Dwarf|               8|       3|Helms Deep|
|Gandalf|Wizard|              12|       1|     Moria|
|Boromir| Human|               7|       4|  Amon Hen|
|Legolas|   Elf|              20|       0|  Amon Hen|
|Aragorn| Human|               9|       2|  Amon Hen|
+-------+------+----------------+--------+----------+



In [30]:
# group by race to calculate the average enemies defeated,
# and order by the average in descending order. 
result_df_2 = (
    df_filter_2.groupBy("race")
    .agg(
        avg("enemies_defeated").alias("avg_enemy_Defeated"))
    .orderBy(col("avg_enemy_Defeated").desc()))

result_df_2.show()

+------+------------------+
|  race|avg_enemy_Defeated|
+------+------------------+
|   Elf|              17.5|
|Wizard|              12.0|
| Human| 8.666666666666666|
| Dwarf|               8.0|
+------+------------------+



In [31]:
data = [
    ("SUB-01", "Pacific Strike", 5, "Success"),
    ("SUB-02", "Atlantic Surge", 2, "Failure"),
    ("SUB-01", "Arctic Blitz", 4, "Success"),
    ("SUB-03", "Indian Ocean", 6, "Success"),
    ("SUB-02", "Pacific Strike", 3, "Success"),
    ("SUB-01", "Coral Sea", 7, "Success"),
    ("SUB-03", "Arctic Blitz", 1, "Failure"),
    ("SUB-02", "Bering Strait", 5, "Success")
]

columns = ["submarine_id", "mission_name", "warheads_launched", "status"]
df = spark.createDataFrame(data, columns)
df.show()

+------------+--------------+-----------------+-------+
|submarine_id|  mission_name|warheads_launched| status|
+------------+--------------+-----------------+-------+
|      SUB-01|Pacific Strike|                5|Success|
|      SUB-02|Atlantic Surge|                2|Failure|
|      SUB-01|  Arctic Blitz|                4|Success|
|      SUB-03|  Indian Ocean|                6|Success|
|      SUB-02|Pacific Strike|                3|Success|
|      SUB-01|     Coral Sea|                7|Success|
|      SUB-03|  Arctic Blitz|                1|Failure|
|      SUB-02| Bering Strait|                5|Success|
+------------+--------------+-----------------+-------+



In [34]:
# Select the submarine ID, mission name, and warheads launched, 
# filter for missions where more than 3 warheads were launched successfully, 
# group by submarine ID to count the total missions and sum the warheads launched, 
# and order by total warheads launched in descending order. 

df_filter_3 = df.filter((col("status") == "Success" ) & (col("warheads_launched") > 3))
df_filter_3.show()

+------------+--------------+-----------------+-------+
|submarine_id|  mission_name|warheads_launched| status|
+------------+--------------+-----------------+-------+
|      SUB-01|Pacific Strike|                5|Success|
|      SUB-01|  Arctic Blitz|                4|Success|
|      SUB-03|  Indian Ocean|                6|Success|
|      SUB-01|     Coral Sea|                7|Success|
|      SUB-02| Bering Strait|                5|Success|
+------------+--------------+-----------------+-------+



In [35]:
from pyspark.sql.functions import count
result_df_3 = (
    df_filter_3.groupBy("submarine_id")
    .agg(count("mission_name").alias("total_mission"))
    .orderBy(col("total_mission").desc()))
result_df_3.show()

+------------+-------------+
|submarine_id|total_mission|
+------------+-------------+
|      SUB-01|            3|
|      SUB-03|            1|
|      SUB-02|            1|
+------------+-------------+

